In [ ]:
# | default_exp _code_generator.test_generator

In [ ]:
# | export

from typing import *
import time

from yaspin import yaspin
from fastkafka._components.logger import get_logger

In [ ]:
from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

SAMPLE_CODE = """
import asyncio
from fastkafka.testing import Tester
from application import *

async def async_tests():
    async with Tester(kafka_app).using_inmemory_broker() as tester:
        input_msg = StoreProduct(
            product_name="Mobile Phone",
            currency="HRK",
            price=750.0
        )

        # tester produces message to the store_product topic
        await tester.to_store_product(input_msg)

        # assert that app consumed from the store_product topic and it was called with the accurate argument
        await kafka_app.awaited_mocks.on_store_product.assert_called_with(
            input_msg, timeout=5
        )
        # assert that tester consumed from the change_currency topic and it was called with the accurate argument
        await tester.awaited_mocks.on_change_currency.assert_called_with(
            StoreProduct(
                product_name="Mobile Phone",
                currency="EUR",
                price=100.0
            ), timeout=5
        )
    print("ok")


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())
"""

In [ ]:
# | export

def generate_test(app_code: str) -> str:
    """Generate test for the new FastKafka app
    
    Args:
        app_code: The generated application code
        
    Returns:
        The generated test code for the application
    """
    # TODO: Implement the actual functionality
    with yaspin(text="Generating tests...", color="cyan", spinner="clock") as sp:

        time.sleep(3)
        sp.text = ""
        sp.ok(" ✔ Tests are generated and saved at: /some_dir/test.py")
        return SAMPLE_CODE

In [ ]:
code = """
from pydantic import BaseModel, Field, NonNegativeFloat

from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class Data(BaseModel):
    data: NonNegativeFloat = Field(
        ..., example=0.5, description="Float data example"
    )

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Demo Kafka app",
    kafka_brokers=kafka_brokers,
)

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: Data):
    logger.info(f"Got data: {msg.data}")
    await to_output_data(msg.data)


@kafka_app.produces(topic="output_data")
async def to_output_data(data: float) -> Data:
    processed_data = Data(data=data+1.0)
    return processed_data
"""

test = generate_test(code)
print(test)

⠹ Generating tests... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:59: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


✔ Tests generated!    

import asyncio
from fastkafka.testing import Tester
from application import *

async def async_tests():
    async with Tester(kafka_app).using_inmemory_broker() as tester:
        input_msg = StoreProduct(
            product_name="Mobile Phone",
            currency="HRK",
            price=750.0
        )

        # tester produces message to the store_product topic
        await tester.to_store_product(input_msg)

        # assert that app consumed from the store_product topic and it was called with the accurate argument
        await kafka_app.awaited_mocks.on_store_product.assert_called_with(
            input_msg, timeout=5
        )
        # assert that tester consumed from the change_currency topic and it was called with the accurate argument
        await tester.awaited_mocks.on_change_currency.assert_called_with(
            StoreProduct(
                product_name="Mobile Phone",
                currency="EUR",
                price=100.0
         